# 1. Extraction of data
The first thing to do is extracting the data we are interested in from the XML files (XML files are used from a dataset of US patent applications from 2001 to 2016. This dataset can be found here: https://figshare.com/articles/dataset/Chemical_reactions_from_US_patents_1976-Sep2016_/5104873). We are interested in the title, the general experimental procedure, the reaction SMILES, but most importantly the product SMILES.
Therefore, using xml.Etree in the following we iterate through the child elements of the root of the file. 


In [7]:
# First we import the necessary libraries

import xml.etree.cElementTree as et     # for parsing the XML file
import pandas as pd
import numpy as np
import os
import re
from groq import Groq # for the LLM Groq queries (needs to be installed via pip)
import json
from tqdm import tqdm
from config import LLM_API_KEY
from tqdm import tqdm

Now we use a function to iterate over all the XML files in the applications data from 2001. We collect the title, the paragraphText, mostly including the experimental procedures, the reaction SMILES and the product SMILES from it.

In [3]:
def extract_data(root_file):
    """function that extracts data from an XML file and returns a list of dictionaries containing the extracted information.
        Information to be extracted includes the title of the reaction, the experimental procedure, the reaction SMILES, and the product SMILES.

    Args:
        root_file: root of the parsed XML file

    Returns:
        reaction_list: list of dictionaries containing the extracted information
    """
    
    # Define the namespace that prevents mismatching of tags in the XML file
    ns = {'cml': 'http://www.xml-cml.org/schema', 'dl': 'http://bitbucket.org/dan2097'}

    # Create lists to store extracted information
    reaction_list = []

    # Find all <reaction> elements
    reaction_elements = root_file.findall('.//cml:reaction', ns)

    # Iterate over each <reaction> element
    for reaction_element in reaction_elements:
        # Create a dictionary to store information about the reaction
        reaction_dict = {}

        # Extract title
        title = reaction_element.find('.//dl:headingText', ns)
        if title is not None:
            reaction_dict['title'] = title.text

        # Extract paragraph text
        paragraph_text = reaction_element.find('.//dl:paragraphText', ns)
        if paragraph_text is not None:
            reaction_dict['paragraphText'] = paragraph_text.text

        # Extract reaction SMILES
        reaction_smiles = reaction_element.find('.//dl:reactionSmiles', ns)
        if reaction_smiles is not None:
            reaction_dict['reactionSmiles'] = reaction_smiles.text

        # Extract product SMILES
        product_elements = reaction_element.findall('.//cml:product', ns)
        product_smiles = []
        for product_element in product_elements:
            identifier_element = product_element.find('.//cml:identifier[@dictRef="cml:smiles"]', ns)
            if identifier_element is not None:
                smiles_value = identifier_element.get('value')
                if smiles_value is not None:
                    product_smiles.append(smiles_value)
        if product_smiles:
            reaction_dict['productSmiles'] = product_smiles

        # Append the reaction dictionary to the reaction list
        reaction_list.append(reaction_dict)

    # Check if any values were extracted
    #print("Reaction List:", reaction_list)
    return reaction_list


In a second step, we iterate through all files in every folder of the application data of the dataset and extract the data with the function extract_data. This takes quite a bit, but you can see the progress whenever a new folder is treated. 

In [3]:
Applications_list = []

# define path to access first XML file in the folder 2001 of applications
# Insert the path to the folder containing the XML files
for folder in os.listdir(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications'):
    folder = os.path.join(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications', folder)
    print(folder) 
    for file in os.listdir(folder):
        if file.endswith('.xml'):
            file = os.path.join(folder, file)
            tree = et.parse(file)
         # define root of the XML file to iterate through the file
            root = tree.getroot()
            Applications_list.append(extract_data(root))    
                  

C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2001
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2002
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2003
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2004
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2005
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2006
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2007
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2008
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2009
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2010


KeyboardInterrupt: 

# 2. Create a dataframe from extracted data.
Now that we created a list of lists containing a dictionnary for every reaction that was extracted from the XML files, we put the lists into a dataframe.

In [ ]:
# Create a DataFrame from the extracted data
df_extracts = pd.DataFrame(Applications_list[0])

# Iterate through the list of dictionaries and create a DataFrame
for i in range(1, len(Applications_list)):
    df = pd.DataFrame(Applications_list[i])
    
    # Concatenate the DataFrames
    df_extracts = pd.concat([df_extracts, df], ignore_index=True)

print(df_extracts.shape)
df_extracts.head()


(1939253, 4)


,paragraphText,reactionSmiles,productSmiles,title
0,"PL 137,526 describes the hydrogenation of p-te...",[C:1]([C:5]1[CH:10]=[CH:9][C:8]([OH:11])=[CH:7...,[C(C)(C)(C)C1CCC(CC1)O],NaN
1,"Slurry aluminum chloride (140.9 g, 1.075 mol) ...",[Cl-].[Al+3].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C(C)C],Step h: 4-Chloro-1-(4-isopropyl-phenyl)-butan-...
2,"Suspend anhydrous AlCl3 (156 g, 1.15 mol) in t...",[Al+3].[Cl-].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C],Step d: 4-Chloro-1-(4-methyl-phenyl)-butan-1-one
3,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...
4,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...


That Dataframe is huge! Finally, save all the extracted data into a csv file on the local device.

In [ ]:
df_extracts.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv', index=False)

# 3. Pre-Processing of the Data using Regex and a LLM
Of course, not all entries in the data frame can be used for our model. Many of the experimental procedures do not include any information about the Rf value. Thus, we need to find the entries with Rf values. There is only one problem: all experimental procedures are written differently. Thus, filtering them with Regex is quite a pain and did not give the outcome, we envisioned. To circumvent this, we created a function, that pre-filters the experimental procedure to drop the rows that do not include any information about Rf values and keep the other entries. Those remaining entries are treated with a LLM (to get it to run, you have to insert your API key for Groq) and the response is parsed with regex again to extract the information we need. 

In [2]:
def extract_rf_eluent(Dataframe: pd.DataFrame):
    """Function that applies defined regex patterns to data in a dataframe and creates 
    the following new columns: 
    Rf value, solvent A, solvent B, % solvent A, % solvent B. If no Rf value can be found, all columns
    are filled with NaN. 

    Args:
        Dataframe (_type_): Dataframe containing the extracted data from the US patents
    """
    # copy the dataframe to leave old dataframe unchanged
    df = Dataframe.copy()
    
    # Define the regex patterns
    Rf_check = r'( ?R[fF]?[ :=(]?)'
    Rf_pattern = r'(0\.(?!0|9)\d{1,2})\b(?! *mg\b| *mL\b| *g\b)' # exclude decimals that start by 0.0x, this yet seems not to work though, try to specifiy that digit after . cannot start with 0.
    #Rf_pattern = r'[ =:]?(0\.[^0][0-9])[^\dmglL](?! mg)' # exclude decimals that start by 0.0x, this yet seems not to work though, try to specifiy that digit after . cannot start with 0.
    
    #set a count to see how many multiple Rf values are found in the dataframe, how many NaN values are found
    count = 0
    count_nan = 0
    
    # Extract the Rf values from the paragraphText and put them into a new column
    for index, row in df.iterrows():
        checkRf = re.findall(Rf_check, row['paragraphText'])
        
        #check if Rf value can be found in the paragraphText column
        if checkRf:
             #try to find the Rf value in the paragraphText column
            match = re.findall(Rf_pattern, row['paragraphText'])
            if match:
                df.at[index, 'Rf_value'] = match[0] # df['paragraphText'].str.extract(Rf_pattern)
            
                # Check if multiple Rf values were found (potential error source)
                if len(match) > 1:
                    print('Multiple Rf values found in paragraphText:', match, 'at index:', index)
                    count += 1
       
        else:
            df.at[index, 'Rf_value'] = np.nan
            count_nan += 1
            
    print("Number of entries with multiple Rf values:", count)
    print("Number of entries with no Rf values found:", count_nan)
    print("Number of entries with Rf values found:", df['Rf_value'].count())
    return df
    

In [3]:
def extract_rows_with_rf(Dataframe: pd.DataFrame):
    """Function that extracts rows with Rf values from a dataframe and returns a new dataframe containing only these rows. 

    Args:
        Dataframe (_type_): Dataframe containing the extracted data from the US patents
    """
    # copy the dataframe to leave old dataframe unchanged
    df = Dataframe.copy()
    
    # Define the regex patterns
    Rf_check = r'( ?R[fF][ :=(]?)'
    
    # List to store indices of rows without Rf values
    rows_to_drop = []
    
    # Search for rows with Rf values in the paragraphText column
    for index, row in df.iterrows():
        checkRf = re.findall(Rf_check, row['paragraphText'])

        if not checkRf:
            rows_to_drop.append(index)
               
    # Drop rows without Rf values
    df = df.drop(rows_to_drop)
            
    return df
    

In [11]:
def get_values(Dataframe: pd.DataFrame):
    """Function that extracts the Rf value from a row in a Dataframe using LLM and returns it.

    Args:
        DataFrame (_type_): DataFrame containing the extracted data from the US patents
    """
    
    client = Groq(
        api_key= LLM_API_KEY ) # insert your API Key for Groq in the config.py file to use the LLM model
    
    #with open(csv_output_LLM, 'a', newline = '') as csvfile:
     #   writer = csv.writer(csvfile)
    
    for index, row in tqdm(Dataframe.iterrows(), total=Dataframe.shape[0]):
        user_prompt = f"Give me the Rf value, the solvent mixture and the ratio of the solvents in percent for each solvent of this procedure: {row['paragraphText']} in the following format only: Rf= , solvent A= , solvent B= , percent A= , percent B= . If there is a third solvent, please provide the information in the same format, call it additive C = and percent C = . If no information is given for one of the categories, put None for this category. Only give the information requested as output, no additional notes or information!!!" # f"Please only provide the Rf value, solvents A and B, additives C (if applicable), and their ratios in percent as a Python dictionary only (without 'Here is the python dictionary') with keys 'Rf', 'solvent A', 'solvent B', 'additive C', 'percent A', 'percent B', and 'percent C' of this procedure: {row['paragraphText']}. If there is no information for one category, put 'Nan'. Do not provide any additional notes or information except for the dictionary!!! Always use this format!!" # "Give me the Rf value, the solvent A, B and additive C if applied and their ratios in percent as a python dictionary calling the keys Rf (should be a number), solvent A (should be a solvent only), solvent B (solvent only), percent A (number), percent B (number), additive C (solvent), percent C (number) of this procedure:" + row['paragraphText'] + "If there is no information for one category of solvent, put Nan. Do not put any other notes or information except for the dictionary!!! Always use this format!!"  # "Give me the Rf value, the solvent mixture and their ratio of the following procedure" + row['paragraphText'] +  "in the following format of a dictionary only: Rf= , solvent A= , solvent B= , percent A= , percent B= . If there is a third solvent, please provide the information in the same format, call it additive C and percent C = . Only give the values requested as output, no additional notes or information!!!"        
        
        try:             
            # Call the LLM model to extract the Rf value
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
                model="llama3-70b-8192", # other models: LlaMA3 70 b (llama3-70b-8192), mixtral (mixtral-8x7b-32768) can be found here: https://console.groq.com/docs/models
            )
            
            # Extract the Rf value from the response
            response = chat_completion.choices[0].message.content
            #print(response)
            
            # track the progress of the function 
            if index % 100 == 0:
                print("Progress:", (index+1)/len(Dataframe) * 100, "%" )
            
            # Parse the response to extract the Rf value, solvent A, solvent B, % solvent A, and % solvent B using the parse_response function
            rf_value, solvent_a, solvent_b, percent_a, percent_b, additive_c, percent_c = parse_response(response)
            
            # Add extracted values to new columns in the dataframe row
            Dataframe.at[index, 'Rf'] = rf_value
            Dataframe.at[index, 'Solvent_A'] = solvent_a
            Dataframe.at[index, 'Solvent_B'] = solvent_b
            Dataframe.at[index, 'Percent_A'] = percent_a
            Dataframe.at[index, 'Percent_B'] = percent_b
            Dataframe.at[index, 'Additive_C'] = additive_c
            Dataframe.at[index, 'Percent_C'] = percent_c
            
            
            
        except Exception as e:
            # Print the error message and the index of the row where the error occurred
            print(e)
            print("Error at index:", index)
            continue
        
    return Dataframe
        

        
    

In [5]:
def parse_response(response: str):
    """Function that extracts the Rf value, solvent A, solvent B, % solvent A, and % solvent B from a LLM response.

    Args:
        response (str): response from the LLM model 

    Returns:
        rf_value (str)
        solvent_a (str) 
        solvent_b (str) 
        percent_a (str)
        percent_b (str)
        additive_c (str)
        percent_c (str)
    """
    try: 
        
    # search for Rf value in the response using regex
        rf_value_match = re.search(r"Rf\s*=\s*(\d+\.\d+)", response)
        solvent_a_match = re.search(r"solvent\s+A\s*=\s*([^,]+)", response)
        solvent_b_match = re.search(r"solvent\s+B\s*=\s*([^,]+)", response)
        additive_c_match = re.search(r"additive\s+C\s*=\s*([^,]+)", response)
        percent_a_match = re.search(r"percent\s+A\s*=\s*(\d+)", response)
        percent_b_match = re.search(r"percent\s+B\s*=\s*(\d+)", response)
        percent_c_match = re.search(r"percent\s+C\s*=\s*(\d+\.\d+|\d+)", response)
        
    except KeyError:
        raise KeyError(f"Error extracting values from the response: {response_str}")
    
    
    
    # Extract values from regex matches
    rf_value = rf_value_match.group(1) if rf_value_match else None
    solvent_a = solvent_a_match.group(1) if solvent_a_match else None
    solvent_b = solvent_b_match.group(1) if solvent_b_match else None
    percent_a = percent_a_match.group(1) if percent_a_match else None
    percent_b = percent_b_match.group(1) if percent_b_match else None
    additive_c = additive_c_match.group(1) if additive_c_match else None
    percent_c = percent_c_match.group(1) if percent_c_match else None

    # Return the extracted values
    return rf_value, solvent_a, solvent_b, percent_a, percent_b, additive_c, percent_c

First, we load the extracted raw data into a new dataframe.

In [4]:
# That is the csv file that contains all the raw data from the US patents
df_new = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv')
df_new.shape

(1939253, 4)

Now we apply the first function `extract_rows_with_rf` to prefilter the dataframe. Already by this first filter, the size of the datafram shrinks quite a bit! 

In [16]:
df_rf = extract_rows_with_rf(df_new)
df_rf.shape
#df_rf.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\extracted_data_second_processing_rf_values.csv', index=False)
df_rf.head()

,paragraphText,reactionSmiles,productSmiles,title
524,A mixture of 7-trifluoromethylsulfonyloxy-1-(4...,FC(F)(F)S(O[C:7]1[CH:16]=[C:15]2[C:10]([CH:11]...,['CN1CCN(CC1)C1=CC=CC2=CC=C(C=C12)C=1C=NC=NC1'],1-(4-Methylpiperazin-1-yl)-7-(pyrimid-5-yl)nap...
525,"A mixture of the above acid (0.25 g, 0.82 mmol...",[CH3:1][N:2]1[CH2:7][CH2:6]O[CH2:4][CH2:3]1.O....,['ClC1=CC=C(CNC(=O)C2=CC3=C(C=CC=C3C=C2)N2CCN(...,8-(4-Methylpiperazin-1-yl)naphthalene-2-carbox...
538,"A solution of 1,4-benzodioxan-6-amine (2.00 g,...",[O:1]1[C:6]2[CH:7]=[CH:8][C:9]([NH2:11])=[CH:1...,['O1CCOC2=C1C=CC(=C2)N2CCNCC2'],NaN
539,A solution of 6-aminochroman from part 2 of th...,[NH2:1][C:2]1[CH:3]=[C:4]2[C:9](=[CH:10][CH:11...,['O1CCCC2=CC(=CC=C12)N1CCNCC1'],1-(Chroman-6-yl)piperazine
540,"A solution of 1-(1,4-benzodioxan-6-yl)piperazi...",O1C2C=CC(N3CCNCC3)=CC=2OCC1.C(O)(=O)C(O)=O.[O:...,['O1CCOC2=C1C=CC(=C2)N2CCN(CC2)CC2=CC=C(C=C2)F'],NaN


In [8]:
# if we restart from here, we can directly load the dataframe with the extracted rows containing Rf values.

df_rf = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\preprocessed_data_big_dataset_extracted_rows_36579.csv')

For the LLM we use the API of the open source model offered by Groq Clouds (https://console.groq.com/docs/quickstart). Different models can be tested out. This is a first test to get familiar with how it works and to show some examples.

In [21]:
#Access token croq: 
from config import LLM_API_KEY
# Create a Groq client (it is recommended to use the following Quickstart: https://console.groq.com/docs/quickstart)
# However, this did not work in our case and we had to use the following code to create a client 
client = Groq(
    api_key=LLM_API_KEY, # insert your API key here
)
user_prompt =  "Give me the Rf value, the solvent mixture and their ratio of the following procedure in the following format of only: Rf= , solvent A= , solvent B= , percent A= , percent B= . If there is a third solvent, please provide the information in the same format, call it additive C = and percent C = . Only give the information requested as output, no additional notes or information!!!" # "Give me the Rf value, the solvent mixture and their ratio of the following procedure in the following format of a dictionary only: Rf= , solvent A= , solvent B= , percent A= , percent B= . If there is a third solvent, please provide the information in the same format, call it additive C and percent additive C = . Only give the dictionnary as output, no additional notes or information!!!"
procedure = "An Rf value of 0.22 was found using DCM/EtOAc 20:1 and 0.4% Hydroxylammonium. "#"Rf(Hex/EtOAc 1:20):0.22"
user_prompt_procedure = user_prompt + procedure
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": user_prompt_procedure
        }
    ],
    model="mixtral-8x7b-32768", # other models: LlaMA3 70 b (llama3-70b-8192) can be found here: https://console.groq.com/docs/models
)

response_str = chat_completion.choices[0].message.content

print(response_str)
type(response_str)

# response_dict = json.loads(response_str)
#type(response_dict)
#print(response_dict["Rf"])


Rf = 0.22, Solvent A = DCM, Solvent B = EtOAc, Percent A = 95, Percent B = 5, Additive C = 0.4% Hydroxylammonium, Percent C = N/A.


str

As the entire dataframe so far is still too big and it takes too long to compute everything, we cut down the dataframe to a smaller one.

multiprocessing package für parallel computing
delay parallel

In [13]:
# Cut down the dataframe to a smaller size to test the functions
df_rf = df_rf.iloc[:5]
df_rf.shape

(5, 11)

In [14]:
df_processed = get_values(df_rf)
df_processed.head()

 20%|██        | 1/5 [00:00<00:02,  1.45it/s]

Progress: 20.0 %


100%|██████████| 5/5 [00:55<00:00, 11.11s/it]


,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C
0,A mixture of 7-trifluoromethylsulfonyloxy-1-(4...,FC(F)(F)S(O[C:7]1[CH:16]=[C:15]2[C:10]([CH:11]...,['CN1CCN(CC1)C1=CC=CC2=CC=C(C=C12)C=1C=NC=NC1'],1-(4-Methylpiperazin-1-yl)-7-(pyrimid-5-yl)nap...,0.15,ethyl acetate,methanol,80,20,None,None
1,"A mixture of the above acid (0.25 g, 0.82 mmol...",[CH3:1][N:2]1[CH2:7][CH2:6]O[CH2:4][CH2:3]1.O....,['ClC1=CC=C(CNC(=O)C2=CC3=C(C=CC=C3C=C2)N2CCN(...,8-(4-Methylpiperazin-1-yl)naphthalene-2-carbox...,0.28,methylene chloride,methanol,92,8,ammonium hydroxide,0.04
2,"A solution of 1,4-benzodioxan-6-amine (2.00 g,...",[O:1]1[C:6]2[CH:7]=[CH:8][C:9]([NH2:11])=[CH:1...,['O1CCOC2=C1C=CC(=C2)N2CCNCC2'],NaN,0.37,methanol,methylene chloride,10,90,None,None
3,A solution of 6-aminochroman from part 2 of th...,[NH2:1][C:2]1[CH:3]=[C:4]2[C:9](=[CH:10][CH:11...,['O1CCCC2=CC(=CC=C12)N1CCNCC1'],1-(Chroman-6-yl)piperazine,0.35,methanol,methylene chloride,10,90,None,None
4,"A solution of 1-(1,4-benzodioxan-6-yl)piperazi...",O1C2C=CC(N3CCNCC3)=CC=2OCC1.C(O)(=O)C(O)=O.[O:...,['O1CCOC2=C1C=CC(=C2)N2CCN(CC2)CC2=CC=C(C=C2)F'],NaN,0.35,ethyl acetate,hexane,30,70,None,None


In [45]:
df_processed.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\extracted_data_processed_LLM.csv', index=False)

In [25]:
df_processed.head(20)

,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C
524,A mixture of 7-trifluoromethylsulfonyloxy-1-(4...,FC(F)(F)S(O[C:7]1[CH:16]=[C:15]2[C:10]([CH:11]...,['CN1CCN(CC1)C1=CC=CC2=CC=C(C=C12)C=1C=NC=NC1'],1-(4-Methylpiperazin-1-yl)-7-(pyrimid-5-yl)nap...,0.15,ethyl acetate,methanol,80,20,None,None
525,"A mixture of the above acid (0.25 g, 0.82 mmol...",[CH3:1][N:2]1[CH2:7][CH2:6]O[CH2:4][CH2:3]1.O....,['ClC1=CC=C(CNC(=O)C2=CC3=C(C=CC=C3C=C2)N2CCN(...,8-(4-Methylpiperazin-1-yl)naphthalene-2-carbox...,0.28,methylene chloride,methanol,92,7,ammonium hydroxide,0.04
538,"A solution of 1,4-benzodioxan-6-amine (2.00 g,...",[O:1]1[C:6]2[CH:7]=[CH:8][C:9]([NH2:11])=[CH:1...,['O1CCOC2=C1C=CC(=C2)N2CCNCC2'],NaN,0.37,methanol,methylene chloride,10,90,None,None
539,A solution of 6-aminochroman from part 2 of th...,[NH2:1][C:2]1[CH:3]=[C:4]2[C:9](=[CH:10][CH:11...,['O1CCCC2=CC(=CC=C12)N1CCNCC1'],1-(Chroman-6-yl)piperazine,0.35,methanol,methylene chloride,10,90,None,None
540,"A solution of 1-(1,4-benzodioxan-6-yl)piperazi...",O1C2C=CC(N3CCNCC3)=CC=2OCC1.C(O)(=O)C(O)=O.[O:...,['O1CCOC2=C1C=CC(=C2)N2CCN(CC2)CC2=CC=C(C=C2)F'],NaN,0.35,ethyl acetate,hexane,30,70,None,None
543,"To a stirred solution of 4e (355 mg, 0.891 mmo...",[CH2:1]([O:4][C:5]([C:7]1[CH:12]=[CH:11][C:10]...,['O=C1N(C(CC1)=O)OC(C1=CC=C(C=C1)N1N=C(C2=CC=C...,NaN,0.20,cyclohexane,ethyl acetate,66,33,None,None
544,5.295 g of N-chlorosuccinimide is added at 60°...,[Cl:1]N1C(=O)CCC1=O.[C:9]1([CH2:15][O:16][C:17...,['ClC1=C2CC[C@H]3[C@@H]4CCC([C@@]4(C)C[C@@H](C...,Stage A: 4-chloro-11beta-[4-(phenylmethoxy) ph...,0.19,ethyl acetate,cyclohexane,30,70,None,None
545,6 ml of sulphuryl chloride at 10% in dichlorom...,S(Cl)([Cl:4])(=O)=O.[CH3:6][N:7]([CH3:37])[CH2...,['ClC1=C2CC[C@H]3[C@@H]4CCC([C@@]4(C)C[C@@H](C...,Stage A: 4-chloro-11beta-[4-[2-(dimethylamino)...,0.35,ethyl acetate,triethylamine,80,20,None,None
580,5-fluoro-2-methyl-1-(p-methylsulfonylbenzylide...,C[O-].[Na+].[F:4][C:5]1[CH:6]=[C:7]2[C:11](=[C...,['FC=1C=C2C(=C(C(C2=CC1)=CC1=CC=C(C=C1)S(=O)(=...,NaN,0.21,chloroform,methyl isobutyl ketone,80,20,None,None
774,Boc-Lys(Cbz)-OH (25 g) was dissolved in dichlo...,[NH:1]([C:21]([O:23][C:24]([CH3:27])([CH3:26])...,['N([C@@H](CCCCNC(=O)OCC1=CC=CC=C1)C(=O)OC)C(=...,NaN,0.79,ethyl acetate,heptane,75,25,None,None


## Side Note
We decided to cut the dataset into four pieces to reduce computation time by calculating one set at the time. That's what will happen in the next kernels. 

In [6]:
df_4 = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\AfterRFfilter(4).csv')

In [ ]:
df_4_processed = get_values(df_4)
df_4_processed.head()

# 4. Further Processing
Now we created new rows for the extracted values. In the following, we need to process the data further, to first drop the rows that don't include an Rf and were not filtered out in the first function, to convert the types from `str` to `int` for the Rf values and check, if the percent add up to 100 %, to see which solvents are used to create their corresponding SMILES and later molecular descriptors using `RDkit`. Furthermore, in some procedures, a third additive is added to the solvent mixture, which distorts the data and we found out, that the calculation of the solvent ratio in percent is not matching. To simplify the dataset, we thus decided to filter out those entries with a third additive. Lastly, the LLM partly took retention times from column chromatography as Rf values, so to filter out those, we exclude Rf values >1.

In [43]:
def clean_up(Dataframe: pd.DataFrame):
    """1. Delets all row whitout Rf value or Rf value over 1.0 and without solvent information.
       2. Converts percentage of solvents in a Dataframe from str to float and check if they add up to 100, else drop them.
       3. Strips productSMILES str to a usable SMILES str.

    Args: 
        Dataframe (_type_): Dataframe containing the processed data from the get_value function.
        Dataframe needs to have following columns: 'productSmiles,' 'Rf', 'Percent_A', 'Percent_B'

    """
    
    # Drop rows without Rf values
    Dataframe.dropna(subset=['Rf'], inplace = True) 
    
    # Convert Rf value to float
    Dataframe['Rf'] = Dataframe['Rf'].astype(float)
    
    #finds indicies with Rf values over 1.0
    indices_false_Rf = Dataframe[Dataframe["Rf"] > 1].index 
    Dataframe.drop(indices_false_Rf,inplace = True) #drops rows with false Rf values
    
    #convert 'None' entry to 0
    Dataframe['Percent_A'] = Dataframe['Percent_A'].apply(lambda x: 0 if x is None else x)
    
    #convert 'None' entry to 0
    Dataframe['Percent_B'] = Dataframe['Percent_B'].apply(lambda x: 0 if x is None else x)
    
    # convert Percentage to float
    Dataframe['Percent_A'] = Dataframe['Percent_A'].apply(lambda x: float(x)) 
    
    #convert Percantage to float
    Dataframe['Percent_B'] = Dataframe['Percent_B'].apply(lambda x: float(x)) 
    
    # remove [' and '] from the productSimles
    Dataframe['productSmiles'] = Dataframe['productSmiles'].apply(lambda x: x[2:-2]) 
    
    # check if the sum of the percentages is 100, this at the same time kicks out entries with additives C
    Dataframe['sum'] = Dataframe['Percent_A'] + Dataframe['Percent_B']
    Dataframe = Dataframe[Dataframe['sum'] == 100]
    
    Dataframe.reset_index(drop=True, inplace=True) # sets the index new from 1 to end

    return Dataframe


In [27]:
def get_solvents(Dataframe: pd.DataFrame):
    """Function that iterates over all solvent entries in the dataframe and checks, if the solvent mentioned is already in the list. If not, it adds the solvent to the list.

    Args:
        Dataframe (_type_): dataframe with the extracted data from the US patents, preprocessed with the get_values function.
    """
    solvents = []
    
    for index, row in Dataframe.iterrows():
        solvent_a = row['Solvent_A']
        solvent_b = row['Solvent_B']
        
        if solvent_a not in solvents:
            solvents.append(solvent_a)
        if solvent_b not in solvents:
            solvents.append(solvent_b)
        else:
            continue
        
    return solvents

In [44]:
def convert_solvent(Dataframe: pd.DataFrame):
    """Converts the solvent names in the dataframe to a standardized form.

    Args:
        Dataframe (pd.DataFrame): Dataframe containing the extracted data from the US patents, 
                                preprocessed with the get_values and clean_up function.
    """
    
    for index, row in Dataframe.iterrows():
        if 'dcm' or 'dichlorometh' or 'methylene chloride' or 'ch2cl2' in row['Solvent_A'].lower():
            Dataframe.at[index, 'Solvent_A'] = 'DCM'
        elif row['Solvent_A'] == 'EtOAc':
            Dataframe.at[index, 'Solvent_A'] = 'Ethyl acetate'
        elif row['Solvent_A'] == 'MeOH':
            Dataframe.at[index, 'Solvent_A'] = 'Methanol'
        elif row['Solvent_A'] == 'MeCN':
            Dataframe.at[index, 'Solvent_A'] = 'Acetonitrile'
        elif row['Solvent_A'] == 'Toluene':
            Dataframe.at[index, 'Solvent_A'] = 'Toluene'
        elif row['Solvent_A'] == 'Hexane':
            Dataframe.at[index, 'Solvent_A'] = 'Hexane'
        else:
            Dataframe.at[index, 'Solvent_A'] = 'Other'
        
        if row['Solvent_B'] == 'DCM':
            Dataframe.at[index, 'Solvent_B'] = 'Dichloromethane'
        elif row['Solvent_B'] == 'EtOAc':
            Dataframe.at[index, 'Solvent_B'] = 'Ethyl acetate'
        elif row['Solvent_B'] == 'MeOH':
            Dataframe.at[index, 'Solvent_B'] = 'Methanol'
        elif row['Solvent_B'] == 'MeCN':
            Dataframe.at[index, 'Solvent_B'] = 'Acetonitrile'
        elif row['Solvent_B'] == 'Toluene':
            Dataframe.at[index, 'Solvent_B'] = 'Toluene'
        elif row['Solvent_B'] == 'Hexane':
            Dataframe.at[index, 'Solvent_B'] = 'Hexane'
        else:
            Dataframe.at[index, 'Solvent_B'] = 'Other'

In [42]:
def normalize_data(Dataframe: pd.DataFrame):
    """Normalizes the percentage of solvents in the dataframe for machine learning to get values between 0 and 1 as for the Rf values.

    Args:
        Dataframe (pd.DataFrame): Dataframe containing the extracted data from the US patents, 
                                preprocessed with the get_values and clean_up function.
    """
    
    for index, row in Dataframe.iterrows():
        Dataframe.at[index, 'Percent_A'] = row['Percent_A'] / 100
        Dataframe.at[index, 'Percent_B'] = row['Percent_B'] / 100
        
    return Dataframe

In [38]:
df_processed_cleaned = clean_up(df_processed)
df_processed_cleaned.head()

,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C,sum
0,A mixture of 7-trifluoromethylsulfonyloxy-1-(4...,FC(F)(F)S(O[C:7]1[CH:16]=[C:15]2[C:10]([CH:11]...,1CCN(CC1)C1=CC=CC2=CC=C(C=C12)C=1C=NC=N,1-(4-Methylpiperazin-1-yl)-7-(pyrimid-5-yl)nap...,0.15,ethyl acetate,methanol,80.0,20.0,None,None,100.0
1,"A solution of 1,4-benzodioxan-6-amine (2.00 g,...",[O:1]1[C:6]2[CH:7]=[CH:8][C:9]([NH2:11])=[CH:1...,CCOC2=C1C=CC(=C2)N2CCNC,NaN,0.37,methanol,methylene chloride,10.0,90.0,None,None,100.0
2,A solution of 6-aminochroman from part 2 of th...,[NH2:1][C:2]1[CH:3]=[C:4]2[C:9](=[CH:10][CH:11...,CCCC2=CC(=CC=C12)N1CCNC,1-(Chroman-6-yl)piperazine,0.35,methanol,methylene chloride,10.0,90.0,None,None,100.0
3,"A solution of 1-(1,4-benzodioxan-6-yl)piperazi...",O1C2C=CC(N3CCNCC3)=CC=2OCC1.C(O)(=O)C(O)=O.[O:...,CCOC2=C1C=CC(=C2)N2CCN(CC2)CC2=CC=C(C=C2,NaN,0.35,ethyl acetate,hexane,30.0,70.0,None,None,100.0
4,5.295 g of N-chlorosuccinimide is added at 60°...,[Cl:1]N1C(=O)CCC1=O.[C:9]1([CH2:15][O:16][C:17...,C1=C2CC[C@H]3[C@@H]4CCC([C@@]4(C)C[C@@H](C3=C2...,Stage A: 4-chloro-11beta-[4-(phenylmethoxy) ph...,0.19,ethyl acetate,cyclohexane,30.0,70.0,None,None,100.0


In [39]:
df_processed_cleaned.shape

(1638, 12)

In [40]:
solvents = get_solvents(df_processed_cleaned)
print(solvents)
print(len(solvents))

['ethyl acetate', 'methanol', 'methylene chloride', 'hexane', 'cyclohexane', 'triethylamine', 'chloroform', 'methyl isobutyl ketone', 'heptane', 'n-butanou', 'acetic acid', 'Heptane', 'Ethyl acetate', 'ether', 'None', 'EtOAc', 'MeOH', 'CH2Cl2', 'CHCL3', 'CH3OH', 'CHCl3', 'hexanes', 'Ethyl Acetate', 'Hexane', 'dichloromethane', 'pentane', 'Hexanes', 'Methanol', 'NH4OH', 'hex', 'concentrated aqueous ammonia', None, 'toluene', 'silica gel', 'Dichloromethane', 'acetonitrile', 'chloroform\nsolvent B= methanol\npercent A= 30-40\npercent B= 70-60', 'methanol\npercent A= 30-40\npercent B= 70-60', ' ', 'PE', 'system B', 'ethanol', 'FM1', 'FM4', 'FM 4', 'Triethylamine', 'Hex', 'EtoAc', 'acetone', 'acetic ester', 'aqueous ammonia', 'Et2O', 'Chloroform', 'CH2CL2', 'ethylacetate', 'DCM', 'light petroleum ether', 'light petroleum', 'Acetone', 'Toluene', 'EtOAC', 'n-hexane', 'CH2Cl', 'CH2CI2', 'CH', 'EA', 'Et3N', 'benzene', 'water', 'ethanol acetate', 'Cyclohexane', 'AcOEt', 'ethyl', 'acetate', '5% s

In [36]:
"pentane" in solvents

True